# MySQL connection

In [ ]:
import mysql.connector
from mysql.connector import Error
from sqlalchemy import create_engine
import pymysql

## 1. Functions for MySQL connections and queries
### 1.1. Connection to MySQL Server

In [ ]:
# Function to connect to MySQL server
def server_connect(host_name, user_name, user_password): # Define function and arguments 
    connection = None # Initialise connection variable and close any existing connections    
    try: # Exception handling in case connection fails 
        connection = mysql.connector.connect( 
            host = host_name,
            user = user_name, 
            password = user_password 
        )
        print(f"Connection to MySQL server {host_name} successful") # If connection is successful message will appear
        return connection
    except Error as err:
        print(f"Error: '{err}'")

### 1.2. Create new database

In [ ]:
# Function to create a new database using the above connection and a query as parameters
def create_database(connection, query):
    cursor = connection.cursor() # Create a cursor object
    try:
        cursor.execute(query) # Use cursor to execute the query 
        connection.commit() # Commit changes
        print("Database created successfully") # Print message if successful
    except Error as err:
        print(f"Error: '{err}'") # Print error message if unsuccessful

In [ ]:
# Define credentials to use as parameters in function
user = "root"
host = "localhost"
pw = input("Enter MySQL terminal password:")
db = input("Enter database name:")

connection = server_connect("localhost", "root", pw) # Call the function with the above credentials as parameters and assign to connection
create_database_query = f"CREATE DATABASE IF NOT EXISTS {db}" # Define query
create_database(connection, create_database_query) # Call function using connection and create_database_query

### 1.3. Connnect to database

In [ ]:
# Function to connect to a MySQL database using credentials as parameters 
def db_connect(host_name, user_name, user_password, db_name):
    connection = None  
    try:
        connection = mysql.connector.connect( # Use credentials and database name defined in previous sections as parameters
            host = host_name,
            user = user_name,
            password = user_password,
            database = db_name
        )
        print(f"Connection to MySQL database {db_name} successful")
    except Error as err:
        print(f"Error: '{err}'")
        
    return connection # Return connection object defined in previous section

### 1.4. Execute query

In [ ]:
def execute_query(connection, query, values=None):
    cursor = connection.cursor() # Create cursor object from the connection
    try:
        if isinstance(values, list): 
            cursor.executemany(query, values) # If value is list use executemany  
        else:
            cursor.execute(query, values) # Else use execute for single set of values
            connection.commit() # Commit changes to database 
        print("Query successful")
    except Error as err:
        print(f"Error: '{err}'")
    finally:
        cursor.close() # CLose the cursor

### 1.5. Load data into database

In [ ]:
# pymysql is used to improve performance with the data upload and prevent losing connection to the server 
# Chunks used to save on memory
def load_data(dataframe, database, table, chunk_size=500000): 
    engine = create_engine(f'mysql+pymysql://{user}:{pw}@{host}/{db}') # Create a database engine with the credentials 
    
    # Splitting the dataframe into chunks and uploading each chunk
    for start_row in range(0, dataframe.shape[0], chunk_size):
        end_row = start_row + chunk_size
        df_chunk = dataframe.iloc[start_row:end_row]
        
        df_chunk.to_sql(name=table, con=engine, if_exists='append', index=False)  # Load the chunk into the table
        print(f"Chunk {start_row} to {end_row} successfully imported into {table}")

### 1.6. Read queries

In [ ]:
# Additional function required to read queries as opposed to the execute_function
def read_query(connection, query):
    cursor = connection.cursor() # Create cursor object
    result = None # Initialise and clear result variable 
    try:
        cursor.execute(query) # execute query
        result = cursor.fetchall() # Fetch all rows that are returned from the query
        return result
    except Error as err:
        print(f"Error: '{err}'")